# Attention Ensemble  - Survival(맞춘 모델만 살아남기)
---

### **Import Libraries**

In [203]:
import tensorflow as tf
from keras.models import Model
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split 

import unicodedata
import re
import numpy as np
import os
import io
import time
import random
import openpyxl

### **데이터 로드**

In [204]:
path_to_file_esb = '/Users/ahjeong_park/Study/Attention-Ensemble-Translation/spa-eng/spa_for_esb.txt'

### **데이터(문장) 전처리**

In [205]:
# 유니코드 파일을 아스키 코드 파일로 변환합니다.
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # 단어와 단어 뒤에 오는 구두점(.)사이에 공백을 생성합니다.
  # 예시: "he is a boy." => "he is a boy ."
  # 참고:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # (a-z, A-Z, ".", "?", "!", ",")을 제외한 모든 것을 공백으로 대체합니다.
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # 모델이 예측을 시작하거나 중단할 때를 알게 하기 위해서
  # 문장에 start와 end 토큰을 추가합니다.
  w = '<start> ' + w + ' <end>'
  return w

### **Dataset 생성**
1. 문장에 있는 억양을 제거합니다.
2. 불필요한 문자를 제거하여 문장을 정리합니다.
3. 다음과 같은 형식으로 문장의 쌍을 반환합니다: [영어, 스페인어]

In [206]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

### **Language 가 들어오면 공백 단위로 토큰화**
- fit_on_texts(): 문자 데이터를 입력받아서 리스트의 형태로 변환
- texts_to_sequences: 텍스트 안의 단어들을 숫자 시퀀스로 출력
- pad_sequcences(tensor, padding='post') : 서로 다른 개수의 단어로 이루어진 문장을 같은 길이로 만들어주기 위해 패딩을 사용
  - padding = 'post' : [[ 0  0  0  5  3  2  4], [ 0  0  0  5  3  2  7],...,]
  - padding = 'pre' : 뒤 부터 패딩이 채워짐
  - 가장 긴 sequence 의 길이 만큼
  

In [207]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

### **전처리된 타겟 문장과 입력 문장 쌍을 생성**
- input_tensor : input 문장의 패딩 처리된 숫자 시퀀스
- inp_lang_tokenizer : input 문장을 공백 단위로 토큰화, 문자 -> 리스트 변환
- target_tensor, targ_lang_tokenizer : 위와 비슷


In [208]:
def load_dataset(path, num_examples=None):
  
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### **언어 데이터셋 크기 제한**
- 언어 데이터셋을 아래의 크기로 제한하여 훈련과 검증을 수행
- inp_lang, targ_lang : 인풋,타겟 문장의 문자 -> 리스트 변환 결과
- max_length_targ, max_length_inp : 인풋, 타겟 문장의 '패딩된' 숫자 시퀀스 길이 -> 타겟 텐서와 입력 텐서의 최대 길이

In [209]:
# num_examples = 30000
num_examples = 60000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file_esb, num_examples)

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

### **데이터셋 (테스트 & 검증) 분리**

In [210]:
# 훈련 집합과 검증 집합을 80대 20으로 분리합니다.
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# 훈련 집합과 검증 집합의 데이터 크기를 출력합니다.
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

48000 48000 12000 12000


### 인덱스 -> 해당 word 로

```
Input Language; index to word mapping
1 ----> <start>
93 ----> tomas
27 ----> le
1063 ----> escribio
7 ----> a
120 ----> maria
3 ----> .
2 ----> <end>
```


```
Target Language; index to word mapping
1 ----> <start>
8 ----> tom
695 ----> wrote
6 ----> to
31 ----> mary
3 ----> .
2 ----> <end>
```



In [211]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

### **Buffer, Batch, epoch, embedding dimension, units 설정**
- Tokenizer 의 word_index 속성 : 속성은 단어와 숫자의 키-값 쌍을 포함하는 딕셔너리를 반환
- 따라서 vocab_inp_size, vocab_inp_size : 인풋, 타겟의 단어-숫자 딕셔너리 최대 길이 + 1 (?)
- dataset.batch(BATCH_SIZE, drop_remainder = True) : 배치사이즈 만큼 분할 후 남은 데이터를 drop 할 것인지 여부
- shuffle : 데이터셋 적절히 섞어준다.

In [212]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

# 훈련 집합에서만 shuffle, batch
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [213]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 53]), TensorShape([64, 51]))

### **Encoder**


1.   초기화 : vocab_size(단어의 크기), embedding_dim(임베딩 차원 수), enc_units(인코더의 히든 사이즈), batch_sz(배치 사이즈)
  - embedding_dim : 단어 -> 임베딩 벡터로 하기 위한 차원 수
2.  call : gru 에 들어가 output, state 출력
3.  initialize_hidden_state : 맨 처음 gru에 들어가기 위한 더미 입력 값




In [214]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

### **Encoder 객체 생성**

In [215]:
# encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

### **Attention**


In [216]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # 쿼리 은닉 상태(query hidden state)는 (batch_size, hidden size)쌍으로 이루어져 있습니다.
    # query_with_time_axis은 (batch_size, 1, hidden size)쌍으로 이루어져 있습니다.
    # values는 (batch_size, max_len, hidden size)쌍으로 이루어져 있습니다.
    # 스코어(score)계산을 위해 덧셈을 수행하고자 시간 축을 확장하여 아래의 과정을 수행합니다.
    query_with_time_axis = tf.expand_dims(query, 1)

    # score는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다.
    # score를 self.V에 적용하기 때문에 마지막 축에 1을 얻습니다.
    # self.V에 적용하기 전에 텐서는 (batch_size, max_length, units)쌍으로 이루어져 있습니다.
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다. 
    attention_weights = tf.nn.softmax(score, axis=1)

    # 덧셈이후 컨텍스트 벡터(context_vector)는 (batch_size, hidden_size)쌍으로 이루어져 있습니다.
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

### **Decoder**


1.   초기화 : vocab_size(단어의 크기), embedding_dim(임베딩 차원 수), enc_units(인코더의 히든 사이즈), batch_sz(배치 사이즈)
2.   encoder 와의 차이점 : 마지막 fully_connected_layer(tf.keras.layers.Dense) 추가



In [217]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # 어텐션을 사용합니다.
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output는 (batch_size, max_length, hidden_size)쌍으로 이루어져 있습니다.
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # 임베딩층을 통과한 후 x는 (batch_size, 1, embedding_dim)쌍으로 이루어져 있습니다.
    x = self.embedding(x)

    # 컨텍스트 벡터와 임베딩 결과를 결합한 이후 x의 형태는 (batch_size, 1, embedding_dim + hidden_size)쌍으로 이루어져 있습니다.
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # 위에서 결합된 벡터를 GRU에 전달합니다.
    output, state = self.gru(x)

    # output은 (batch_size * 1, hidden_size)쌍으로 이루어져 있습니다.
    output = tf.reshape(output, (-1, output.shape[2]))

    # output은 (batch_size, vocab)쌍으로 이루어져 있습니다.
    x = self.fc(output)

    # return x, state, attention_weights
    return x, state

### **Decoder 객체 생성**

In [218]:
# decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

### **NMT Model 생성**

In [219]:
class NMT_Model():
  def __init__(self):
    super(NMT_Model, self).__init__()
    self.encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
    self.decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

### **Ensemble Model 생성**

In [220]:
models = []
# num_models = 5 
num_models = 15
for m in range(num_models):
  m = NMT_Model()
  models.append(m)


### **Loss Function & Optimizer**

In [221]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

### **Chekcpoint**
- 여기서 학습한 매개변수를 저장, optimizer/encoder/decoder

In [222]:
checkpoint_dir = '/Users/ahjeong_park/Study/Attention-Ensemble-Translation/training_checkpoints_esb'
checkpoint_dir_test = '/Users/ahjeong_park/Study/Attention-Ensemble-Translation/training_checkpoints_esb2'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoints = []

for m in range(num_models):
  checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=models[m].encoder,
                                 decoder=models[m].decoder)
  checkpoints.append(checkpoint)


In [223]:
# print(checkpoints)

### **Train_step**

In [224]:
# @tf.function
def train_step(model, inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = model.encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # 교사 강요(teacher forcing) - 다음 입력으로 타겟을 피딩(feeding)합니다.
    for t in range(1, targ.shape[1]):
      # enc_output를 디코더에 전달합니다.
      predictions, dec_hidden = model.decoder(dec_input, dec_hidden, enc_output)
      # print('predictions', predictions.shape)

      loss += loss_function(targ[:, t], predictions)

      # 교사 강요(teacher forcing)를 사용합니다. -> 훈련에서는 실제 값을 이용
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))
  variables = model.encoder.trainable_variables + model.decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

### **학습**
- 학습 코드 X

### **문장 번역(스페인 -> 영어)** 

*   tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen='', padding='post') : 일정한 길이(maxlen)로 맞춰준다. (패딩은 뒤에서)
*   

  ```
  inp_lang.word_index :  {'<start>': 1, '<end>': 2, '.': 3, 'tom': 4, '?': 5...}
  ```

* tf.expand_dims: 차원을 늘려준다.




In [225]:
def evaluate(sentence):

  
  sentence = preprocess_sentence(sentence)

  # 문장, input 딕셔너리 출력 
  print ('sentence:', sentence)

  no_word = 'no word'

  input_list = []
   
  for j in range(num_models):
    inp = []
    for i in sentence.split(' '):
        if i in inp_lang.word_index:
            inp.append(inp_lang.word_index[i])
        else:
            print('no words!')
            return no_word, sentence
    input_list.append(inp)
    

  for input in range(len(input_list)):
    input_list[input] = tf.keras.preprocessing.sequence.pad_sequences([input_list[input]],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    input_list[input] = tf.convert_to_tensor(input_list[input])
    
  voting_result = ''
  
  hiddens = []
  for i in range(num_models):
    hiddens.append([tf.zeros((1, units))])

  # enc_out, dec_hidden, dec_input 리스트 생성
  enc_outs = []
  enc_hiddens = []
  dec_hiddens = [] 
  dec_inputs = []

  # Encoder 의 hidden 을 Decoder 의 hidden 으로 받는다.
  for i in range(num_models):
    eo, eh = models[i].encoder(input_list[i], hiddens[i])
    enc_outs.append(eo)
    enc_hiddens.append(eh)
    

  for i in range(num_models):
    dec_hiddens.append(enc_hiddens[i])


  # Decoder 의 시작인 '<start>' 
  for i in range(num_models):
    dec_inputs.append(tf.expand_dims([targ_lang.word_index['<start>']], 0))


  # Model Prediction List
#   pred_num = 5
  pred_num = 15
  suvi_models = models.copy()

  # Target 의 최대 길이 만큼 출력
  for t in range(max_length_inp):
    pred_list = []   # 모델 1~5 의 predictions 가 저장되어 있는 리스트
    pred_ids = []    # 모델 1~5의 predictions 에서 가장 큰 값을 가지는 인덱스가 저장되어 있는 리스트
    max_list = []    # 모델 1~5의 predictions 에서 가장 큰 값을 나타내는 리스트
    if pred_num > 2:
        for pred in range(pred_num):

            # 1. 모델(1~5)의 예측과 hidden decoder를 출력한다. --> predictions
            predictions, dec_hidden = suvi_models[pred].decoder(dec_inputs[pred],
                                                             dec_hiddens[pred],
                                                             enc_outs[pred])
            # 2. 모델(1~5)의 예측을 pred_list 리스트에 저장
            #    모델(1~5)의 hidden decoder을 dec_hiddens 리스트에 저장
            pred_list.append(predictions)
            dec_hiddens[pred] = dec_hidden

            # 3. 모델(1~5)의 predictions 에서 가장 큰 softmax 값을 가진 인덱스 번호를 max_idx 에 저장한다.
            #    모델(1~5)에서 가장 높은 확률로 뽑힌 인덱스를 저장하는 리스트에 max_idx 를 추가한다.
            max_idx = tf.argmax(predictions[0]).numpy()
            pred_ids.append(max_idx)

            # 각 모델의 예측에서 가장 큰 확률 값만 모아놓은 리스트
            max_list.append(predictions[0][max_idx])


        # 4. 모델(1~5)의 예측 리스트(pred_ids) 에서 가장 많이 등장한 인덱스번호 찾기
        #    즉 다수결로 뽑힌 단어(semi_answer) 찾기
        semi_answer = max(pred_ids, key=pred_ids.count)

        # 5. 세미정답을 출력한 살아남은 모델 --> winner
        winner = list(np.where(np.array(pred_ids) == semi_answer)[0])
        # 6. 서바이벌 - 제일 나쁜 모델 탈락 --> loser
        loser = list(np.where(np.array(pred_ids) != semi_answer)[0])

        # 7. winner 모델만 다시 서바이벌 참여할 수 있도록 참여모델 업데이트
        update_sm = [] # suvi_models 업데이트
        upd_dech = [] # dec_hiddens 업데이트
        upd_enco = [] # enc_outs 업데이트
        upd_deci = [] # dec_inputs 업데이트

        for win in winner:
            update_sm.append(suvi_models[win])
            upd_dech.append(dec_hiddens[win])
            upd_enco.append(enc_outs[win])
            upd_deci.append(dec_inputs[win])

        # winner 로만 구성된 서바이벌 모델들 --> suvi_models, dec_hiddens, enc_outs
        suvi_models = update_sm 
        dec_hiddens = upd_dech
        enc_outs = upd_enco
        dec_inputs = upd_deci

#         print('참가한 모델의 예측: ', pred_ids)
#         print('다수결 결과: ', semi_answer)
#         print('살아남은 모델: ', winner)
#         print('탈락한 모델: ', loser)
#         print('번역 결과 : ', targ_lang.index_word[semi_answer])
        
        # 8. 다음 예측 for loop 을 위해 탈락한 모델(loser) 개수 제외
        pred_num -= len(loser)

        # 9. 세미정답(semi_answer)에 해당하는 word 저장
        voting_result += targ_lang.index_word[semi_answer] + ' '

        if targ_lang.index_word[semi_answer] == '<end>':
          return voting_result, sentence

        # 예측된 ID(세미정답으로)를 모델에 다시 피드합니다. (semi_answer)
        for i in range(pred_num):
            dec_inputs[i] = tf.expand_dims([semi_answer], 0)
            
    elif pred_num == 2:
        # 살아있는 모델 개수가 2개
        for pred in range(pred_num):
            # 1. 모델(1~5)의 예측과 hidden decoder를 출력한다. --> predictions
            predictions, dec_hidden = suvi_models[pred].decoder(dec_inputs[pred],
                                                             dec_hiddens[pred],
                                                             enc_outs[pred])
            # 2. 모델(1~5)의 예측을 pred_list 리스트에 저장
            #    모델(1~5)의 hidden decoder을 dec_hiddens 리스트에 저장
            pred_list.append(predictions)
            dec_hiddens[pred] = dec_hidden

            # 3. 모델(1~5)의 predictions 에서 가장 큰 softmax 값을 가진 인덱스 번호를 max_idx 에 저장한다.
            #    모델(1~5)에서 가장 높은 확률로 뽑힌 인덱스를 저장하는 리스트에 max_idx 를 추가한다.
            max_idx = tf.argmax(predictions[0]).numpy()
            pred_ids.append(max_idx)

            # 각 모델의 예측에서 가장 큰 확률 값만 모아놓은 리스트
            max_list.append(predictions[0][max_idx])
            
        if pred_ids[0] == pred_ids[1]:
            semi_answer = pred_ids[0]
            
            # 10. 세미정답(semi_answer)에 해당하는 word 저장
            voting_result += targ_lang.index_word[semi_answer] + ' '
            
#             print('참가한 모델의 예측: ', pred_ids)
#             print('다수결 결과: ', semi_answer)
#             print('번역 결과 : ', targ_lang.index_word[semi_answer])

            if targ_lang.index_word[semi_answer] == '<end>':
              return voting_result, sentence

            # 예측된 ID(세미정답으로)를 모델에 다시 피드합니다. (semi_answer)
            for i in range(pred_num):
                dec_inputs[i] = tf.expand_dims([semi_answer], 0)
        else:   # 예측 값이 다를 때
            semi_answer = pred_ids[max_list.index(max(max_list))]
            voting_result += targ_lang.index_word[semi_answer] + ' '
        
#             print('참가한 모델의 예측: ', pred_ids)
#             print('다수결 결과: ', semi_answer)
#             print('번역 결과 : ', targ_lang.index_word[semi_answer])
        
            if targ_lang.index_word[semi_answer] == '<end>':
              return voting_result, sentence
            
            # 이제 모델 1개로 진행한다.
            loser = list(np.where(np.array(pred_ids) != semi_answer)[0])
            for l in loser:
                del suvi_models[l]
                del dec_hiddens[l]
                del enc_outs[l]
                del dec_inputs[l]
                
            pred_num -= 1
            
#             print('탈락한 모델(2개중): ', loser)

            # 예측된 ID(세미정답으로)를 모델에 다시 피드합니다. (semi_answer)
            for i in range(pred_num):
                dec_inputs[i] = tf.expand_dims([semi_answer], 0)
    else:
        # 살아남은 모델 1개
        
        predictions, dec_hiddens[0] = suvi_models[0].decoder(dec_inputs[0],
                                                             dec_hiddens[0],
                                                             enc_outs[0])
        predicted_id = tf.argmax(predictions[0]).numpy() 
        voting_result += targ_lang.index_word[predicted_id] + ' '
        
#         print('최후 1개 모델의 예측: ', predicted_id)
#         print('최후 1개 모델의 번역 결과 : ', targ_lang.index_word[predicted_id])
        
        if targ_lang.index_word[predicted_id] == '<end>':
          return voting_result, sentence
    
        # 예측된 ID를 모델에 다시 피드합니다.
        dec_inputs[0] = tf.expand_dims([predicted_id], 0)
            
        
  return voting_result, sentence

In [226]:
def translate(sentence):
    voting_result, sentence = evaluate(sentence)
    
    
    print('Survival result: ', voting_result)
    
    return voting_result

### **Checkpoint 복원**

In [227]:
ckp_dir_m1 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints1'
ckp_dir_m2 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints2'
ckp_dir_m3 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints3'
ckp_dir_m4 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints4'
ckp_dir_m5 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints5' 
ckp_dir_m6 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints6'
ckp_dir_m7 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints7'
ckp_dir_m8 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints8'
ckp_dir_m9 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints9'
ckp_dir_m10 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints10'
ckp_dir_m11 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints11'
ckp_dir_m12 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints12'
ckp_dir_m13 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints13'
ckp_dir_m14 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints14'
ckp_dir_m15 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints15' 

In [228]:
# checkpoint_dir내에 있는 최근 체크포인트(checkpoint)를 복원
checkpoints[0].restore(tf.train.latest_checkpoint(ckp_dir_m1))
checkpoints[1].restore(tf.train.latest_checkpoint(ckp_dir_m2))
checkpoints[2].restore(tf.train.latest_checkpoint(ckp_dir_m3))
checkpoints[3].restore(tf.train.latest_checkpoint(ckp_dir_m4))
checkpoints[4].restore(tf.train.latest_checkpoint(ckp_dir_m5))
checkpoints[5].restore(tf.train.latest_checkpoint(ckp_dir_m6))
checkpoints[6].restore(tf.train.latest_checkpoint(ckp_dir_m7))
checkpoints[7].restore(tf.train.latest_checkpoint(ckp_dir_m8))
checkpoints[8].restore(tf.train.latest_checkpoint(ckp_dir_m9))
checkpoints[9].restore(tf.train.latest_checkpoint(ckp_dir_m10))
checkpoints[10].restore(tf.train.latest_checkpoint(ckp_dir_m11))
checkpoints[11].restore(tf.train.latest_checkpoint(ckp_dir_m12))
checkpoints[12].restore(tf.train.latest_checkpoint(ckp_dir_m13))
checkpoints[13].restore(tf.train.latest_checkpoint(ckp_dir_m14))
checkpoints[14].restore(tf.train.latest_checkpoint(ckp_dir_m15))

### 이 코드로 했을 때 학습 바로 돌렸을 때와 같은 결과가 나왔음.

### **번역 시작**

In [229]:
# test dataset 파일 불러오기
testdata = '/Users/ahjeong_park/Study/Survival-Ensemble/Dataset/test_data.txt'
start_row = 2

# 엑셀 파일 불러오기
wb = openpyxl.load_workbook('Survival_Translate.xlsx')

# 엑셀 파일의 시트 활성화
sheet1 = wb['Survival(win)']

f = open(testdata, 'r')
lines = f.readlines()

for sentence in lines:
    result = translate(sentence)
    
    sheet1.cell(row = start_row, column = 4).value = result

    start_row += 1
    
f.close()
wb.save('Survival_Translate.xlsx')

sentence: <start> te dijeron lo que paso , ¿ no ? <end>
Survival result:  you told you what happened , don t you ? <end> 
sentence: <start> ellos no estaban alla . <end>
Survival result:  they were not there . <end> 
sentence: <start> no me gusta ninguno de los chicos . <end>
Survival result:  i don t like any of boys . <end> 
sentence: <start> jamas trabaje con el . <end>
Survival result:  i never expected him . <end> 
sentence: <start> mi padre cultiva arroz . <end>
Survival result:  my father drinks rice . <end> 
sentence: <start> dime que hiciste en shounan . <end>
Survival result:  tell me what you did to shounan . <end> 
sentence: <start> ¿ cuantas palabras deberias escribir ? <end>
Survival result:  how many of you should write to ? <end> 
sentence: <start> ¿ como te introdujiste en mi casa ? <end>
Survival result:  how did you get in my house ? <end> 
sentence: <start> mi madre hornea pan todas las mananas . <end>
Survival result:  my mother bakes bread every morning . <end> 
s

Survival result:  this work is light enough for me to read . <end> 
sentence: <start> nunca he ido al extranjero . <end>
Survival result:  i ve never gone abroad . <end> 
sentence: <start> comamos mientras este caliente . <end>
Survival result:  let s eat while you this hot . <end> 
sentence: <start> odio estar soltero . <end>
Survival result:  i hate to be single . <end> 
sentence: <start> no os preocupeis por mi . <end>
Survival result:  don t worry about me . <end> 
sentence: <start> ella me evita . <end>
Survival result:  she helps me . <end> 
sentence: <start> me desperte tres veces durante la noche . <end>
Survival result:  i woke up three times during the night . <end> 
sentence: <start> el nino cogio un pez grande . <end>
Survival result:  the boy caught a large fish . <end> 
sentence: <start> tom perdio sus gafas . <end>
Survival result:  tom lost his glasses . <end> 
sentence: <start> la puerta se abrio y un hombre entro . <end>
Survival result:  the door opened and a man cam

Survival result:  i think watching tv is a waste of time . <end> 
sentence: <start> la reina elizabeth i fallecio en . <end>
no words!
Survival result:  no word
sentence: <start> el ha escrito muchas historias . <end>
Survival result:  he wrote a lot of stories . <end> 
sentence: <start> de todos los libros publicados ultimamente , solo algunos merecen ser leidos . <end>
no words!
Survival result:  no word
sentence: <start> necesito regresar . <end>
Survival result:  i need to go back . <end> 
sentence: <start> probate este pullover . <end>
no words!
Survival result:  no word
sentence: <start> tom actua como si ni conociera a maria . <end>
Survival result:  tom acted as if he didn t know mary . <end> 
sentence: <start> es mi ultima oferta . <end>
Survival result:  it s my last offer . <end> 
sentence: <start> no me voy a casar nunca . <end>
Survival result:  i m not going to get married . <end> 
sentence: <start> no necesitas ir tan apurado . <end>
Survival result:  you don t need to g

Survival result:  you ll regret it ! <end> 
sentence: <start> hay una necesidad urgente de dinero . <end>
Survival result:  there is an urgent need for money . <end> 
sentence: <start> tienen que ser estadounidenses . <end>
Survival result:  they must be americans . <end> 
sentence: <start> no tuvimos muchas visitas este verano . <end>
Survival result:  we didn t have much time this summer . <end> 
sentence: <start> todos los camaradas dormian . <end>
no words!
Survival result:  no word
sentence: <start> tom tiene miedo al compromiso . <end>
Survival result:  tom is afraid of commitment . <end> 
sentence: <start> eso no es de tu incumbencia . <end>
Survival result:  that s none of your business . <end> 
sentence: <start> queremos ir de boston a chicago . <end>
Survival result:  we want to go to boston and chicago . <end> 
sentence: <start> ella se decidio por el abrigo rojo . <end>
Survival result:  she made up her coat on her red . <end> 
sentence: <start> creia que eras mayor . <end>

Survival result:  do you know how to cook well ? <end> 
sentence: <start> no queremos causar ningun problema . <end>
Survival result:  we don t want to cause any trouble . <end> 
sentence: <start> le puse algo de leche a mi cafe . <end>
Survival result:  i put some milk to my coffee . <end> 
sentence: <start> este texto esta destinado a los principiantes . <end>
Survival result:  this text is bound to the th . <end> 
sentence: <start> ¿ quien te conto la historia ? <end>
Survival result:  who told you the story ? <end> 
sentence: <start> le vemos todos los dias . <end>
Survival result:  i ll meet him every day . <end> 
sentence: <start> ellos no cederan . <end>
no words!
Survival result:  no word
sentence: <start> tom estaba muy complacido con los resultados . <end>
no words!
Survival result:  no word
sentence: <start> si no come , morira . <end>
Survival result:  if you don t eat , try . <end> 
sentence: <start> ayudemonos los unos a los otros . <end>
no words!
Survival result:  no wo

...

Survival result:  the exhibition would be the perfect match for me if i could the steam a foreign language that they are dealing with nuts and view is vanilla . <end> 
sentence: <start> nosotros soliamos dar una caminata antes del desayuno . <end>
Survival result:  we used to take a walk before breakfast . <end> 
sentence: <start> tenemos que ir al colegio . <end>
Survival result:  we have to go to school . <end> 
sentence: <start> tom tiene una reunion todos los lunes por la tarde . <end>
Survival result:  tom has a meeting every morning . <end> 
sentence: <start> las pruebas de mediado de ano estan a la vuelta de la esquina . <end>
no words!
Survival result:  no word
sentence: <start> mientras haya esperanza , existe la posibilidad . <end>
Survival result:  the silence , the wind following chance . <end> 
sentence: <start> se que es dificil hablar al respecto . <end>
Survival result:  i know it s difficult to talk about it . <end> 
sentence: <start> desconozco si tom nadara o no . <e

Survival result:  it may snow this evening . <end> 
sentence: <start> almorcemos fuera . <end>
Survival result:  let s have been outside . <end> 
sentence: <start> me gustaria retirar algun dinero en efectivo . <end>
Survival result:  i d like to take some money in cash . <end> 
sentence: <start> el gato salto sobre la mesa . <end>
Survival result:  the cat jumped over the table . <end> 
sentence: <start> contacte a mis padres . <end>
Survival result:  keep my parents . <end> 
sentence: <start> solo somos amigos . <end>
Survival result:  we re just friends . <end> 
sentence: <start> ten cuidado con ese cuchillo . <end>
Survival result:  be careful with that knife . <end> 
sentence: <start> el camino va paralelo al rio . <end>
no words!
Survival result:  no word
sentence: <start> ella era una mujer de mediana edad . <end>
Survival result:  she was a woman of beautiful age . <end> 
sentence: <start> ¿ que conoces de australia ? <end>
Survival result:  what do you know about australia ? <

Survival result:  why can t you join you ? <end> 
sentence: <start> me temo que no hay mucho que pueda hacer para ayudar . <end>
Survival result:  i m afraid there s not much left that i can do to help . <end> 
sentence: <start> creo que estamos listos para ir alli . <end>
Survival result:  i think we re ready to go there . <end> 
sentence: <start> la tormenta no mostro senales de abatirse . <end>
no words!
Survival result:  no word
sentence: <start> ya paso lo peor . <end>
Survival result:  it s already worse . <end> 
sentence: <start> quiero que entiendas lo que estoy tratando de decir . <end>
Survival result:  i want you to understand what i m trying to say . <end> 
sentence: <start> la chica habla muy bien ingles . <end>
Survival result:  the girl speaks very well well . <end> 
sentence: <start> me gusta mas el invierno que el verano . <end>
Survival result:  i like the summer than summer . <end> 
sentence: <start> ¿ donde esta la clase de tom ? <end>
Survival result:  where s tom 

Survival result:  some people were a lot of the most part of time . <end> 
sentence: <start> cristobal colon descubrio america . <end>
Survival result:  columbus discovered america . <end> 
sentence: <start> a finales de agosto , las fuerzas aliadas tomaron paris . <end>
no words!
Survival result:  no word
sentence: <start> vendre a verte el domingo a las tres de la tarde . <end>
Survival result:  i ll come and see you at school at p . m . <end> 
sentence: <start> quiero que seas mi socio . <end>
Survival result:  i want you to be my business . <end> 
sentence: <start> ella es amable . <end>
Survival result:  she is kind . <end> 
sentence: <start> no esta de acuerdo con su familia . <end>
Survival result:  he s not agree with his family . <end> 
sentence: <start> esta ocurriendo esta noche . <end>
Survival result:  this is going off this evening . <end> 
sentence: <start> deberias haber llamado a la policia . <end>
Survival result:  you should ve called the police . <end> 
sentence: <s

Survival result:  he is not the kind of the class . <end> 
sentence: <start> esta no sera la ultima vez . <end>
Survival result:  this won t be the last time . <end> 
sentence: <start> el acaba de comprarse un coche usado . <end>
Survival result:  he just bought a car an old man . <end> 
sentence: <start> despues de conferenciar varias horas hicieron publicos los acuerdos . <end>
no words!
Survival result:  no word
sentence: <start> no puedo creer que tom no venga . <end>
Survival result:  i can t believe tom won t come . <end> 
sentence: <start> ¿ que te pareceria una taza de te u otra cosa , si es que no estas apurado ? <end>
no words!
Survival result:  no word
sentence: <start> el millonario tenia la intencion de adquirir la obra maestra sin importar lo que costara . <end>
Survival result:  the millionaire had intended to carry the masterpiece no matter what it would cost it was making it was making it was making it was making it was making it was making it was making it was making 

In [230]:
# translate('¿Tiene bebidas sin alcohol?')

In [231]:
# translate(u'hace mucho frio aqui.')  # it s very cold here

In [232]:
# translate(u'¿todavia estan en casa?')  # Are you still at home?

In [233]:
# translate(u'¿Con quién desea hablar?')  # Who do you want to talk to?

In [234]:
# translate(u'No puedo creer que Tom no venga.')

In [235]:
# translate(u'esta es mi vida.')  # this is my life